In [1]:
import kagglehub
import numpy as np
import pandas as pd
import missingno as msno
from pathlib import Path
from sklearn.linear_model import LogisticRegression

# data viz
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")

100%|██████████| 639M/639M [00:06<00:00, 98.6MB/s]


Extracting files...


'/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1'

In [5]:
path = Path("/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1")

In [72]:
data_path = path / 'User Listening History.csv'
users_path = path / 'Music Info.csv'

In [81]:
data_song = pd.read_csv(users_path)

In [82]:
data_user = pd.read_csv(data_path)

In [83]:
data_song.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [84]:
data_user.head()

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [85]:
data_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9711301 entries, 0 to 9711300
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   track_id   object
 1   user_id    object
 2   playcount  int64 
dtypes: int64(1), object(2)
memory usage: 222.3+ MB


In [86]:
data_user.duplicated(subset = ['track_id', 'user_id', 'playcount']).sum()

np.int64(0)

There are **0 duplicate rows** in the dataset.

In [87]:
data_user.isna().sum()

,0
track_id,0
user_id,0
playcount,0


In [88]:
print(f"There are {data_user['track_id'].nunique()} unique tracks in the dataset.")

print(f"There are {data_user['user_id'].nunique()} unique users in the dataset.")

print(f"There are {data_user['playcount'].nunique()} unique playcount in the dataset.")

There are 30459 unique tracks in the dataset.
There are 962037 unique users in the dataset.
There are 430 unique playcount in the dataset.


In [89]:
# vf
track_play_counts = data_user['track_id'].value_counts()

In [90]:
top_10_track_ids_by_count = track_play_counts.head(10)

In [108]:
top_10_track_ids_by_count.sample(5)

,count
track_id,
TRWAQOC12903CB84CA,27222
TRPGPDK12903CCC651,27276
TRGCHLH12903CB7352,29708
TRAALAH128E078234A,26689
TRUFTBY128F93450B8,39529


In [92]:
top_10_songs_by_count = data_song[data_song['track_id'].isin(top_10_track_ids_by_count.index.tolist())]

In [93]:
top_10_songs_by_count.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
44,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,https://p.scdn.co/mp3-preview/3debe861147de110...,0jLnevC3Vn34qVWrAa4X6x,"rock, alternative, indie, pop, alternative_roc...",NaN,1999,358333,0.383,...,9,-5.409,1,0.0405,0.031300,0.000000,0.3470,0.514,171.052,4
59,TRPFYYL128F92F7144,Float On,Modest Mouse,https://p.scdn.co/mp3-preview/0d741d740ea0e449...,1Urf1M52P3R6NYdAOJizoW,"rock, alternative, indie, alternative_rock, in...",Rock,2004,208466,0.646,...,6,-4.799,1,0.0301,0.015100,0.000004,0.0892,0.559,100.951,4
323,TRONYHY128F92C9D11,Revelry,Kings of Leon,https://p.scdn.co/mp3-preview/1c2604b381e6106c...,039Q3UlFQ6kavVIZHpO4mL,"rock, alternative, indie, alternative_rock, in...",Rock,2008,201733,0.437,...,4,-6.593,1,0.0689,0.196000,0.000328,0.1580,0.383,149.834,4
1876,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,https://p.scdn.co/mp3-preview/fec54783c7f773f2...,0naTARZScsZOtx3nlhlq0Y,"rock, alternative, indie, pop, singer_songwrit...",Rock,2010,263280,0.626,...,2,-7.989,1,0.0228,0.197000,0.000497,0.2720,0.297,97.081,4
2107,TRUFTBY128F93450B8,Alejandro,Lady Gaga,https://p.scdn.co/mp3-preview/fc79c08bc9f46cca...,0CXHrBetrvDx4PwBar1ZWj,"electronic, pop, female_vocalists, dance",Pop,2010,274800,0.626,...,11,-6.777,0,0.0435,0.000582,0.001140,0.3740,0.353,98.995,4


In [94]:
track_play_sums = data_user.groupby('track_id')['playcount'].sum()

In [95]:
top_10_track_ids_by_playcount = track_play_sums.sort_values(ascending=False).head(10)

In [96]:
top_10_songs_by_playcount = data_song[data_song['track_id'].isin(top_10_track_ids_by_playcount.index.tolist())]

In [97]:
top_10_songs_by_playcount.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
44,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,https://p.scdn.co/mp3-preview/3debe861147de110...,0jLnevC3Vn34qVWrAa4X6x,"rock, alternative, indie, pop, alternative_roc...",NaN,1999,358333,0.383,...,9,-5.409,1,0.0405,0.031300,0.000000,0.3470,0.514,171.052,4
59,TRPFYYL128F92F7144,Float On,Modest Mouse,https://p.scdn.co/mp3-preview/0d741d740ea0e449...,1Urf1M52P3R6NYdAOJizoW,"rock, alternative, indie, alternative_rock, in...",Rock,2004,208466,0.646,...,6,-4.799,1,0.0301,0.015100,0.000004,0.0892,0.559,100.951,4
323,TRONYHY128F92C9D11,Revelry,Kings of Leon,https://p.scdn.co/mp3-preview/1c2604b381e6106c...,039Q3UlFQ6kavVIZHpO4mL,"rock, alternative, indie, alternative_rock, in...",Rock,2008,201733,0.437,...,4,-6.593,1,0.0689,0.196000,0.000328,0.1580,0.383,149.834,4
1876,TRXWAZC128F9314B3E,Heartbreak Warfare,John Mayer,https://p.scdn.co/mp3-preview/fec54783c7f773f2...,0naTARZScsZOtx3nlhlq0Y,"rock, alternative, indie, pop, singer_songwrit...",Rock,2010,263280,0.626,...,2,-7.989,1,0.0228,0.197000,0.000497,0.2720,0.297,97.081,4
2107,TRUFTBY128F93450B8,Alejandro,Lady Gaga,https://p.scdn.co/mp3-preview/fc79c08bc9f46cca...,0CXHrBetrvDx4PwBar1ZWj,"electronic, pop, female_vocalists, dance",Pop,2010,274800,0.626,...,11,-6.777,0,0.0435,0.000582,0.001140,0.3740,0.353,98.995,4


In [98]:
combined = pd.concat([top_10_songs_by_playcount, top_10_songs_by_count], axis=1)

In [101]:
user_track_counts = data_user.groupby("user_id")["track_id"].count()
most_diverse_users = user_track_counts.sort_values(ascending=False).head(10)

In [102]:
user_play_sums = data_user.groupby("user_id")["playcount"].sum()
most_active_users = user_play_sums.sort_values(ascending=False).head(10)

In [103]:
top_users_summary = pd.concat([most_diverse_users, most_active_users], axis=1)

In [104]:
top_users_summary.columns = ["track_count", "total_playcount"]

In [107]:
top_users_summary.sample(5)

,track_count,total_playcount
user_id,,
b4c94d72b15d3c311c10045a58b31f95d9d12785,357.0,NaN
839223f11c98e0c8017e8ecd6fc7b8706658c966,NaN,1608.0
af3ee32357049dd96231238bd1b019e8142ee6aa,NaN,1579.0
8cb51abc6bf8ea29341cb070fe1e1af5e4c3ffcc,362.0,NaN
6a8a142084a4818c0dcac48bdfb3c39deacf5168,NaN,1942.0
